In [3]:
import requests
from bs4 import BeautifulSoup
import os
import time
# from keras.utils import get_file
from tensorflow.keras.utils import get_file
try:
    from urllib.request import urlretrieve
except ImportError:
    from urllib import urlretrieve
import xml.sax

import subprocess
import re
import mwparserfromhell
import json

In [57]:
index = requests.get('https://dumps.wikimedia.org/enwiki/').text

In [58]:
soup_index = BeautifulSoup(index, 'html.parser')

In [59]:
dumps = [a['href'] for a in soup_index.find_all('a') 
             if a.has_attr('href') and a.text[:-1].isdigit()]
dumps

['20210901/',
 '20210920/',
 '20211001/',
 '20211020/',
 '20211101/',
 '20211120/',
 '20211201/']

In [60]:
sorted(dumps, reverse=True)

['20211201/',
 '20211120/',
 '20211101/',
 '20211020/',
 '20211001/',
 '20210920/',
 '20210901/']

In [65]:
for dump_url in sorted(dumps, reverse=True):
    print(dump_url)
    dump_html = index = requests.get('https://dumps.wikimedia.org/ruwiki/' + dump_url).text
    soup_dump = BeautifulSoup(dump_html, 'html.parser')
    pages_xml = [a['href'] for a in soup_dump.find_all('a') 
                 if a.has_attr('href') and a['href'].endswith('-pages-articles.xml.bz2')]
    if pages_xml:
        break
    time.sleep(0.8)

20211201/


In [66]:
pages_xml

['/ruwiki/20211201/ruwiki-20211201-pages-articles.xml.bz2']

In [67]:
wikipedia_dump = pages_xml[0].rsplit('/')[-1]
url = url = 'https://dumps.wikimedia.org/' + pages_xml[0] 
path = get_file(wikipedia_dump, url)
# path

 609452032/4480678376 [===>..........................] - ETA: 1:20:13

KeyboardInterrupt: 

In [34]:
def process_article(title, text):
    rotten = [(re.findall('\d\d?\d?%', p), re.findall('\d\.\d\/\d+|$', p), p.lower().find('rotten tomatoes')) for p in text.split('\n\n')]
    rating = next(((perc[0], rating[0]) for perc, rating, idx in rotten if len(perc) == 1 and idx > -1), (None, None))
    wikicode = mwparserfromhell.parse(text)
    film = next((template for template in wikicode.filter_templates() 
                 if template.name.strip().lower() == 'infobox film'), None)
    if film:
        properties = {param.name.strip_code().strip(): param.value.strip_code().strip() 
                      for param in film.params
                      if param.value.strip_code().strip()
                     }
        links = [x.title.strip_code().strip() for x in wikicode.filter_wikilinks()]
        return (title, properties, links) + rating

In [35]:
class WikiXmlHandler(xml.sax.handler.ContentHandler):
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buffer = None
        self._values = {}
        self._movies = []
        self._curent_tag = None

    def characters(self, content):
        if self._curent_tag:
            self._buffer.append(content)

    def startElement(self, name, attrs):
        if name in ('title', 'text'):
            self._curent_tag = name
            self._buffer = []

    def endElement(self, name):
        if name == self._curent_tag:
            self._values[name] = ' '.join(self._buffer)

        if name == 'page':
            movie = process_article(**self._values)
            if movie:
                self._movies.append(movie)

In [36]:
# path = 'C:\\Users\\Ольга\\.keras\\datasets\\enwiki-20211120-pages-articles.xml.bz2'
# path = 'enwiki-20211120-pages-articles.xml.bz2'
path = 'enwiki-20211120-pages-articles.xml'

In [37]:
import subprocess

# proc = subprocess.Popen(['echo', 'Hello there'], stdout=subprocess.PIPE)
proc = subprocess.Popen(['echo', 'Hello there'], stdout=subprocess.PIPE, shell=True)
out, err = proc.communicate()
print(out.decode('utf-8'))

"Hello there"



In [38]:
parser = xml.sax.make_parser()
handler = WikiXmlHandler()
parser.setContentHandler(handler)
for line in subprocess.Popen(['bzcat'], stdin=open(path), stdout=subprocess.PIPE, shell=True).stdout:
    print(1)
    try:
        parser.feed(line)
    except StopIteration:
        break

In [39]:
subprocess.Popen(['bzcat'], stdin=open(path), stdout=subprocess.PIPE, shell=True)

In [40]:
with open('wp_movies.ndjson', 'wt') as fout:
    for movie in handler._movies:
         fout.write(json.dumps(movie) + '\n')

In [45]:
path
path = 'enwiki-20211120-pages-articles.xml' #.bz2

In [46]:
import pandas as pd
pd.read_xml(path)

MemoryError: 

In [21]:
!pip install --upgrade pandas --user

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
phik 0.12.0 requires scipy>=1.5.2, but you have scipy 1.5.0 which is incompatible.
You should consider upgrading via the 'c:\programdata\anaconda3\python.exe -m pip install --upgrade pip' command.


In [25]:
!pip install --upgrade pip --user

  Using cached pip-21.3.1-py3-none-any.whl (1.7 MB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\programdata\anaconda3\python.exe -m pip install --upgrade pip' command.


In [10]:
!pip install bz2

ERROR: Could not find a version that satisfies the requirement bz2 (from versions: none)
ERROR: No matching distribution found for bz2


In [13]:
import bz2

with bz2.open(path, 'rt') as handle:
    for line in handle:
        parser.feed(line)

UnicodeDecodeError: 'charmap' codec can't decode byte 0x98 in position 6272: character maps to <undefined>